In [3]:
import time
import os
import torch
import pytorch_mask_rcnn as pmr


# ------------------ adjustable parameters ---------------------

use_cuda = True # choose to use GPU or not
visualize = False # choose to visualize evaluation results or not
val_num_samples = 100 # number of samples during test
dataset = 'coco' # coco or voc
data_dir = 'E:/PyTorch/data/coco2017' # dataset directory
num_classes = 91 # 91 for coco, 21 for voc
ckpt_path = '../checkpoint_coco.pth' # path where to save the checkpoint.pth

# ------------------ adjustable parameters ---------------------

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('cuda: {}\nuse_cuda: {}\n{} GPU(s) available'.format(torch.cuda.is_available(), use_cuda, torch.cuda.device_count()))
print('\ndevice: {}'.format(device))

valset = pmr.datasets(dataset, data_dir, 'val', True, device=device)
classes = valset.classes
coco = valset.coco if hasattr(valset, 'coco') else None

indices = torch.randperm(len(valset)).tolist()
valset = torch.utils.data.Subset(valset, indices[:val_num_samples])

model = pmr.maskrcnn_resnet50(True, num_classes).to(device)

if os.path.exists(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model'])
    
    del checkpoint
    torch.cuda.empty_cache()

cuda: True
use_cuda: True
1 GPU(s) available

device: cuda


In [4]:
since = time.time()

# ------------------ test ---------------------

if coco:
    iou_types = ['bbox', 'segm']
    coco_evaluator = pmr.CocoEvaluator(coco, iou_types)

model.eval()
for (image, target) in valset:
    with torch.no_grad():
        result = model(image)

    if coco:
        result = {k: v.cpu() for k, v in result.items()}
        res = {target['image_id'].item(): result}
        coco_evaluator.update(res)

    if visualize:
        #print('  '.join(classes[l.item()] for l in result['labels']))
        #print('  '.join('{:.2f}'.format(p) for p in result['scores']))
        pmr.show(image, result, classes)

if coco:
    coco_evaluator.synchronize_between_processes()
    coco_evaluator.accumulate()
    coco_evaluator.summarize()

# ------------------ test ---------------------

print('\ntotal time: {:.2f} s'.format(time.time() - since))

IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.366
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.204
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.275
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.317
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.250
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.213
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.335
IoU metric: segm
 Avera